<a href="https://colab.research.google.com/github/stevenAnto/llm/blob/main/LangChaing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Ingresa tu clave de OpenAI: ")


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install langchain langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
llm.invoke("la profesion de IA tiene un futuro prometedor?")

In [ ]:
from  langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un asistente de IA, puedes responder a preguntas sobre IA"),
    ("user","{input}")
])

In [ ]:
chain  = prompt | llm
chain.invoke({"input":"la profesion de IA tiene un futuro prometedor?"})

#Memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI


In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente de {ability}, puedes responder a preguntas sobre {ability}",
     ),
    MessagesPlaceholder(variable_name="history"),#ingresa el historico de la conversacion
    ("human","{input}")
])

In [ ]:
runnable = prompt | model

In [ ]:
!pip install langchain-community


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:
store = {}

In [ ]:
#Para saber si hay contexto anterior
def get_session_history(session_id: str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_message_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"que es una funcion de perdida"},
    config={"configurable":{"session_id":"abc123"}},
    )

In [ ]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"como?, no te entendi"},
    config={"configurable":{"session_id":"abc123"}},
    )

#Tools


In [ ]:
!pip install wikipedia


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tool.name

In [ ]:
tool.description

In [ ]:
tool.args

In [ ]:
tool.run({"query":"Alan Turing"})

In [ ]:
tool.run({"query":"quien es Alan Turing"})

#Agents


In [ ]:
!pip install -U langchain-tavily


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
api_key = getpass(" Ingresa tu Tavily API Key: ")
search = TavilySearchResults(tavily_api_key=api_key)

In [ ]:
search.invoke("Que temperatura hay en Londres")

In [ ]:
tools = [search, tool]

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [ ]:
from langchain import hub

In [ ]:
#prompt de la comunidad
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_tool_calling_agent

In [ ]:
#Creacion de agentes
agent = create_tool_calling_agent(llm,tools,prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [ ]:
agent_executor.invoke({"input":"hola"})

In [ ]:
agent_executor.invoke({"input":"cual es el mejor  superheroe de Marvel y por que?"})

In [ ]:
agent_executor.invoke({"input":"Que temperatura hace en Nueva York?"})

#Ejemplos de Interacciones

In [ ]:
agent_executor.invoke({"input":"Busca quien fue Alan Turing"})

In [ ]:
agent_executor.invoke({"input":"Dime un chiste."})

In [ ]:
agent_executor.invoke({"input":"¿Qué está pasando en el mundo de la inteligencia artificial?"})

In [ ]:
agent_executor.invoke({"input":"Que esta pasando entre EEUU e Iran?"})

In [ ]:
agent_executor.invoke({"input":"Quien fue Alan Turing y que fue lo mas grande que hizo, busca en diversas fuentes como paginas webs,etc"})

In [ ]:
agent_executor.invoke({"input":"Cuentame otro chiste"})

In [ ]:
agent_executor.invoke({"input":"Pero no tienes otros chiste"})

In [ ]:
agent_executor.invoke({"input":"si"})

In [ ]:
agent_executor.invoke({"input":"respondeme tres preguntas, busca en internet 1. ¿Qué otros modelos de colores existen para las imagenes?;¿Qué otros cambios se pueden realizar a las imágenes con OpenCV?; 3. ¿Cuál seria el uso del binary threshold en el procesmiento de imagenes?"})

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})

In [ ]:
agent_executor.invoke({"input":'Cual fue la ultima preegunta que te hice '})

In [ ]:
agent_executor.invoke({"input":'puedes recordar  nuestro chat?'})

In [ ]:
agent_executor.invoke({"input":'que  preguntas te he hecho anteriormente'})

In [ ]:
agent_executor.invoke({"input":'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros'})

In [ ]:
agent_executor.invoke({"input":'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros'})

In [ ]:
agent_executor.invoke({"input":'Tienes acceso al directorio de la computadora donde se esta ejecutando este programa?'})

In [1]:
!pip install langchain-googledrive


In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [4]:
import os
from langchain_googledrive.tools.google_drive.tool import GoogleDriveSearchTool
from langchain_googledrive.utilities.google_drive import GoogleDriveAPIWrapper

In [13]:
os.environ["GOOGLE_ACCOUNT_FILE"] = "/content/miproyectodrive-464014-c08c3b8e7b82.json"
SERVICE_ACCOUNT_FILE = "/content/miproyectodrive-464014-c08c3b8e7b82.json"


In [39]:
folder_id ="1lYk54ck5sSJn5hAGY3XvRwIrXT3msUZr"
#folder_id="root"

In [17]:
import logging

logging.basicConfig(level=logging.INFO)

In [45]:
tool = GoogleDriveAPIWrapper(
    folder_id="1lYk54ck5sSJn5hAGY3XvRwIrXT3msUZr",
    num_results=2,
    template="gdrive-query-in-folder",  # busca dentro de documentos
)


In [22]:
import logging



In [46]:
search_tool = GoogleDriveSearchTool(api_wrapper=tool)

# Búsqueda vacía para listar todo
logging.basicConfig(level=logging.INFO)
resultados = search_tool.run("pruebaDocumento")
print("📁 Archivos encontrados por LangChain:")
print(resultados)

📁 Archivos encontrados por LangChain:
No document found


In [27]:
search_tool.description

"A wrapper around Google Drive Search. Useful for when you need to find a document in google drive. The input should be formatted as a list of entities separated with a space. As an example, a list of keywords is 'hello word'."

In [47]:
# 1. Autenticación
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/drive.metadata.readonly"]
)

# 2. Construir servicio
drive_service = build("drive", "v3", credentials=creds)

# 3. Realizar búsqueda
# Busca archivos dentro del folder que NO estén en la papelera
query = f"'{folder_id}' in parents and trashed = false"

results = drive_service.files().list(
    q=query,
    fields="files(id, name, mimeType, modifiedTime)",
    pageSize=20
).execute()

files = results.get("files", [])
if not files:
    print("❌ No se encontraron archivos.")
else:
    print("✅ Archivos encontrados:")
    for file in files:
        print(f"- {file['name']} ({file['mimeType']}) — modificado: {file['modifiedTime']}")

✅ Archivos encontrados:
- segundoDocumentodePrueba (application/vnd.google-apps.document) — modificado: 2025-06-26T00:47:11.494Z
- pruebaDocumento (application/vnd.google-apps.document) — modificado: 2025-06-25T20:44:38.224Z
